In [159]:
%matplotlib inline
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import cvxopt as opt
from cvxopt import blas, solvers


np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

#Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['MSFT', 'T', 'GOOG']
nAssets = len(tickers)
print("Number of assets: %d"%nAssets)

# Define which online source one should use
data_source = 'yahoo'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2009-02-02'
end_date = '2009-06-30'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
#trainingData = data.DataReader(tickers, data_source, start_date, end_date)['Adj Close']
trainingData = data.get_data_yahoo(tickers, start_date, end_date)

Number of assets: 3


/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [160]:
trainingData = trainingData.sort_index(axis=1)
# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
trainingClose = trainingData.ix['Adj Close']
trainingClose = trainingClose.fillna(method='ffill')
#print(trainingClose)

# Opening prices.
trainingStart = trainingData.ix['Open']
trainingStart = trainingStart.fillna(method='ffill')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [161]:
def calculateReturns(start, close):
    X0 = [start[ticker].iloc[0] for ticker in tickers]
    #print("Starting prices: ", X0)

    Returns = []
    # Closing price every day.
    for i, ticker in enumerate(tickers):
        closingPrices = close[ticker]
        # Caclulate returns
        calcReturn = lambda x: float(x - X0[i]) / X0[i]
        Returns.append(closingPrices.apply(calcReturn))
    return Returns
    
def optimalPortfolioCalc(Returns, retMin):
    # Calculate the expected Return for each asset.
    numDays = len(Returns)
    mus = np.asarray(Returns.mean())
    n = len(mus)
    #print("Mean returns: ", mus)
    mus = np.reshape(mus, (len(mus), 1))

    Cov = opt.matrix(np.cov(Returns.T))
    #print(Cov)
    Mus = opt.matrix(mus)
       
    q = opt.matrix(np.zeros((n, 1)))
    
    # Create constraint matrices
    # w >= 0 and min return constraint.
    G = -opt.matrix(np.concatenate((mus.T, np.eye(n))))   # negative n x n identity matrix
    h = opt.matrix(np.concatenate((-np.ones((1,1))*retMin ,np.zeros((n ,1)))))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    solution = solvers.qp(Cov, q, G, h, A, b)
    portfolio = solution['x']
    
    returns = blas.dot(Mus, portfolio)*numDays
    #print("Returns: ", returns)
    risk = np.sqrt(blas.dot(portfolio, Cov*portfolio))
    #print("Risk : %f"%risk)
    return (portfolio, Mus)

#returns3 = calculateReturns(trainingStart, trainingClose)
#np.mean(returns3, axis=1))
returns = trainingClose.pct_change()
returns = returns.fillna(method='bfill')
retMin = 0.001
optimalPortfolio, Mus = optimalPortfolioCalc(returns, retMin)

print( optimalPortfolio)


[ 3.10e-01]
[ 1.41e-01]
[ 5.49e-01]



In [162]:
def calc_kd(kn_, dn_, close_data):
    n_ticks = len(tickers)
    low = list(close_data.min())
    high = list(close_data.max())

    k_list = [0 for _ in tickers]
    d_list = [0 for i in tickers]

    i = close_data.shape[0]-1
    row = close_data.iloc[i]

    for t in range(n_ticks):
        if high[t] - low[t] == 0:
            rsv = 50
        else:
            rsv = ((row[t] - low[t])*100.0) / (high[t] - low[t])
        k = rsv*(1.0/3) + kn_[t]*(2.0/3)
        k_list[t]  = k    
        d = k*(1.0/3) + dn_[t]*(2.0/3)
        d_list[t]= d
       
    return k_list, d_list


In [163]:
def getTestData():
    start_date = '2009-07-02'
    end_date = '2009-12-31'
    testData = data.get_data_yahoo(tickers, start_date, end_date)
    testData = testData.sort_index(axis=1)


    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    close = testData.ix['Adj Close']
    close = close.fillna(method='ffill')

    # Opening prices.
    start = testData.ix['Open']
    start = start.fillna(method='ffill')
    return (start, close)

testStart, testClose = getTestData()



/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc

In [ ]:
testReturns = testClose.pct_change()
testReturns = testReturns.fillna(method="bfill")

def calculateReturn(weights, returns, riskFreeRate):
    returnsSum = np.sum(returns)
    returns = np.matmul(returnsSum, weights)
    
    # Calculate risk free return
    riskFreeWeight = 1 - np.sum(weights)
    riskFreeReturn = riskFreeWeight * riskFreeRate
    return returns + riskFreeReturn

def getNextWindow(returnRates, newRate):
    newReturnRates = returnRates.drop(returnRates.index[0])
    newReturnRates.append(newRate)
    return newReturnRates

def getSignal(Kn_, Dn_, Kn, Dn):
    "Returns 1 for buy and 0 for sell and -1 for nothing."
    #print(Kn_, Dn_)
    #print(Kn, Dn)
    signals = []
    for i in range(nAssets):
        if Kn_[i] <= Dn_[i] and Kn[i] > Dn[i]:
            signals.append(1)
        elif Kn_[i] >= Dn_[i] and Kn[i] < Dn[i]:
            signals.append(0)
        else:
            signals.append(-1)
    assert(len(signals) == nAssets)
    return signals
    
def calculateTrainingKD():
    K_ = [50 for _ in range(nAssets)]
    D_ = [50 for _ in range(nAssets)]
    K, D = K_, D_
    for i in range(1, len(trainingClose.index)):
        #print(trainingClose.iloc[:i,:])
        K_, D_ = K, D
        K, D = calc_kd(K_, D_, trainingClose.iloc[:i,:])
        #print(K_, D_, K, D)
    return (K_, D_, K, D)

def conservativeStrategy(returnRates, riskFreeRate):
    dailyReturns = []
    close = trainingClose
    K_, D_, K, D = calculateTrainingKD()
    
    for i in range(100):
        optimalPortfolio, Mus = optimalPortfolioCalc(returnRates, retMin)
        testDay = testReturns.iloc[[i]]
        signals = getSignal(K_, D_, K, D)
        portfolio = []
        print(signals)
        
        for i in range(nAssets):
            signal = signals[i]
            if signal:
                portfolio.append(optimalPortfolio[i])
            else:
                portfolio.append(0)
        
        returns = calculateReturn(portfolio, testDay, riskFreeRate)
        dailyReturns.append(returns)
        # Get training Windown by deleting the first row and adding the lates data.
        returnRates = getNextWindow(returnRates, testDay)
        close = getNextWindow(close, testClose.iloc[[i]])
        K_, D_ = K, D
        K, D = calc_kd(K_, D_, close)
    return dailyReturns

def moderateStrategy(returnRates, riskFreeRate):
    dailyReturns = []
    close = trainingClose
    for i in range(10):
        optimalPortfolio, Mus = optimalPortfolioCalc(returnRates, retMin)
        testDay = testReturns.iloc[[i]]
        K, D = calc_kd(close)
        signals = getSignal(K, D)
        portfolio = []
        #print(signals)
       
        # Conservative strategy.
        B = []
        for i in range(nAssets):
            if optimalPortfolio[i] > 0:
                if signals[i]:
                    portfolio.append(1)
                    B.append(1)
                else:
                    portfolio.append(0)
            else:
                portfolio.append(0)
        weight = np.mean(np.asarray(B))
        for i in range(len(portfolio)):
            if portfolio[i]:
                portfolio[i] = weight
                  

        returns = calculateReturn(portfolio, testDay, riskFreeRate)
        dailyReturns.append(returns)
        # Get training Windown by deleting the first row and adding the lates data.
        returnRates = getNextWindow(returnRates, testDay)
        close = getNextWindow(close, testClose.iloc[[i]])
    return dailyReturns
    
def aggressiveStrategy(returnRates, riskFreeRate):
    dailyReturns = []
    close = trainingClose
    K_, D_, K, D = calculateTrainingKD()
    
    for i in range(100):
        optimalPortfolio, Mus = optimalPortfolioCalc(returnRates, retMin)
        testDay = testReturns.iloc[[i]]
        signals = getSignal(K_, D_, K, D)
        portfolio = []
        print(signals)
        
        # Aggressive strategy.
        B = []
        for i in range(nAssets):
            print(i)
            if signals[i] > 0:
                portfolio.append(1)
                B.append(1)
            else:
                portfolio.append(0)

        weight = np.mean(np.asarray(B))
        for i in range(len(portfolio)):
            if portfolio[i]:
                portfolio[i] = weight 
        print("len: ", portfolio)
        returns = calculateReturn(portfolio, testDay, riskFreeRate)
        dailyReturns.append(returns)
        # Get training Windown by deleting the first row and adding the lates data.
        returnRates = getNextWindow(returnRates, testDay)
        close = getNextWindow(close, testClose.iloc[[i]])
        K_, D_ = K, D
        K, D = calc_kd(K_, D_, close)
    return dailyReturns




In [ ]:
riskFreeRate = 0.0001
dailyReturns = aggressiveStrategy(testReturns, riskFreeRate)
#dailyReturns = moderateStrategy(testReturns, riskFreeRate)
#calculateTrainingKD()
#print(dailyReturns)
sumRet = 0
retArray = []
for ret in dailyReturns:
    sumRet += ret
    retArray.append(sumRet)

def getIncrementalReturns(data):
    sumRets = [[0]*nAssets]
    for rets in np.asarray(data):
        sumRet = []
        for i in range(nAssets):
            sumRet.append(sumRets[-1][i] + rets[i])
        sumRets.append(sumRet)
    return sumRets

trainingReturns = trainingClose.pct_change()
trainingReturns = trainingReturns.fillna(method="bfill")
rets = getIncrementalReturns(trainingReturns)
for i in range(nAssets):
    plt.plot(np.asarray(rets)[:, i])
plt.show()
rets = getIncrementalReturns(testReturns)
#print(np.asarray(testReturns))
for i in range(nAssets):
    plt.plot(np.asarray(rets)[:, i])
plt.plot(retArray, c='yellow')
plt.show()

    
print(np.sum(dailyReturns))

In [ ]:
dailyReturns = np.array(dailyReturns)
print('Daily: ',np.mean(dailyReturns), np.std(dailyReturns))
print('Monthly: ',np.mean(dailyReturns)*30, np.std(dailyReturns)*np.sqrt(30))